# Proof of concept of ImViz requirements using glupyter/bqplot

We start off by silencing warnings that can happen when loading data as well as deprecation warnings, for clarity:

In [ ]:
import warnings
warnings.simplefilter('ignore')

Next we import all the required modules/classes/functions:

In [ ]:
import math
from jdaviz import ImViz, ImVizTwoPanel
from ipywidgets import Button, HBox
import matplotlib.pyplot as plt
from echo import delay_callback
from bqplot import Label
from bqplot_image_gl.interacts import MouseInteraction
from glue.plugins.wcs_autolinking.wcs_autolinking import WCSLink

## Basic features

We start off by looking at some of the basic features using a single-image viewer jdaviz app:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')

viewer = imviz.app.get_viewer('viewer-1')
viewer.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer.state.show_axes = False
viewer.state.layers[0].percentile = 99

imviz.app

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [ ]:
viewer.state.layers[0].stretch = 'sqrt'

the colormap:

In [ ]:
viewer.state.layers[0].cmap = plt.cm.viridis

the limits via the percentile option:

In [ ]:
viewer.state.layers[0].percentile = 90

or the limits directly:

In [ ]:
viewer.state.layers[0].v_min = -10
viewer.state.layers[0].v_max = +100

## Mouse over coordinates

By default, there is no overlay showing the current cursor position, but this can be added by making use of the ``MouseInteraction`` class in the bqplot-image-gl package. We set up a new ImViz instance:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')

viewer = imviz.app.get_viewer('viewer-1')
viewer.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer.state.show_axes = False
viewer.state.layers[0].percentile = 99

Next up we set up a bqplot label which we add to the bottom left corner of the plot in orange (we could in fact show this also below/above the plot, or include a background box to ensure the text is readable):

In [ ]:
label = Label(x=[0.05], y=[0.05], text=[''], default_size=12, colors=['orange'])
viewer.figure.marks = viewer.figure.marks + [label]

and we create a callback function which will find the world coordinates of the cursor and the pixel value (note that this is hard-coded to assume the data is 2-d and has a celestial WCS, and that there is just one dataset in the viewer):

In [ ]:
def on_mouse_msg(interaction, data, buffers):
    
    image = imviz.app.data_collection[0]
    
    # Extract data coordinates - these are pixels in the image
    x = data['domain']['x']
    y = data['domain']['y']
    
    overlay = f'x={x:.1f} y={y:.1f}'
    
    # Convert these to a SkyCoord via WCS - note that for other datasets
    # we aren't actually guaranteed to get a SkyCoord out, just for images
    # with valid celestial WCS
    celestial_coordinates = image.coords.pixel_to_world(x, y).icrs.to_string('hmsdms')
    overlay += f' ICRS={celestial_coordinates}'
    
    # Extract data values at this position
    if x > -0.5 and y > -0.5 and x < image.shape[1] and y < image.shape[0]:
        value = image.get_data(image.main_components[0])[int(round(y)), int(round(x))]
        overlay += f' data={value:.2g}'

    # For now we just show the coordinates but it would be easy to show the data
    # values for one or more of the images
    
    if data['event'] == 'mousemove':
        label.text = [overlay]
    elif data['event'] == 'mouseleave':
        label.text = ""
    elif data['event'] == 'mouseenter':
        label.text = ""
        
image = viewer.figure.marks[0]
interaction = MouseInteraction(x_scale=image.scales['x'], y_scale=image.scales['y'], move_throttle=70)
viewer.figure.interaction = interaction
interaction.on_msg(on_mouse_msg)

As you hover over the following image, you should see the overlay in the bottom left:

In [ ]:
imviz.app

## WCS Linking

Another ImViz requirement is the ability to show two images side by side and lock the field of view even if the WCSes are different. For now we implement a simple version of locking which is that every time the user clicks on a position, the field of view in both image viewers is updated to be centred on that position:

In [ ]:
def get_sync_button(viewer1, viewer2):
    
    button_sync = Button(description="Sync field of view")

    def sync_fov(b):
        viewer2.state.x_min = viewer1.state.x_min
        viewer2.state.x_max = viewer1.state.x_max
        viewer2.state.y_min = viewer1.state.y_min
        viewer2.state.y_max = viewer1.state.y_max

    button_sync.on_click(sync_fov)
    
    image = viewer1.figure.marks[0]

    interaction = MouseInteraction(x_scale=image.scales['x'], y_scale=image.scales['y'], move_throttle=70)

    viewer1.figure.interaction = interaction

    def on_mouse_msg(interaction, data, buffers):
        if data['event'] == 'click':
            x = data['domain']['x']
            y = data['domain']['y']
            dx = viewer1.state.x_max - viewer1.state.x_min
            dy = viewer1.state.y_max - viewer1.state.y_min
            with delay_callback(viewer1.state, 'x_min', 'x_max', 'y_min', 'y_max'):
                with delay_callback(viewer2.state, 'x_min', 'x_max', 'y_min', 'y_max'):
                    viewer1.state.x_min = viewer2.state.x_min = x - dx / 2
                    viewer1.state.x_max = viewer2.state.x_max = x + dx / 2
                    viewer1.state.y_min = viewer2.state.y_min = y - dy / 2
                    viewer1.state.y_max = viewer2.state.y_max = y + dy / 2

    interaction.on_msg(on_mouse_msg)
    
    return button_sync

We set up a 2-panel version of ImViz and load in the two datasets:

In [ ]:
imviz = ImVizTwoPanel()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')
imviz.load_data('jw01072001001_01101_00005_nrcb1_cal.fits')

imviz.app.data_collection.add_link(WCSLink(imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]'],
                                           imviz.app.data_collection['jw01072001001_01101_00005_nrcb1_cal[SCI]'] ))

viewer1 = imviz.app.get_viewer('viewer-1')
viewer1.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer1.state.show_axes = False
viewer1.state.layers[0].percentile = 99

viewer2 = imviz.app.get_viewer('viewer-2')
viewer2.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer2.add_data('jw01072001001_01101_00005_nrcb1_cal[SCI]')
viewer2.state.show_axes = False
viewer2.state.layers[0].visible = False
viewer2.state.layers[1].percentile = 99
viewer2.state.reference_data = imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]']

Note that above we essentially add an invisible layer to the second image viewer so that we can select this dataset as a reference dataset in both image viewers, which means that even if the images have different WCS, the field of views will be identical.

In [ ]:
imviz.app

If you click on the following button and click somewhere in the left viewer, the images will both be centred on that position. Note that if you pan/zoom or make a selection you will need to run the following cell again for now so that the interaction gets re-enabled

In [ ]:
button1 = get_sync_button(viewer1, viewer2)
button1

We can also look at an example with very different resolutions and WCS:

In [ ]:
imviz = ImVizTwoPanel()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')
imviz.load_data('2mass_j.fits')

imviz.app.data_collection.add_link(WCSLink(imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]'],
                                           imviz.app.data_collection['2mass_j'] ))

viewer1 = imviz.app.get_viewer('viewer-1')
viewer1.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer1.state.show_axes = False
viewer1.state.layers[0].percentile = 99

viewer2 = imviz.app.get_viewer('viewer-2')
viewer2.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer2.add_data('2mass_j')
viewer2.state.show_axes = False
viewer2.state.layers[0].visible = False
viewer2.state.layers[1].percentile = 99
viewer2.state.reference_data = imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]']

imviz.app

In [ ]:
button1 = get_sync_button(viewer1, viewer2)
button1

## Overlaying data/transparency

A build-in feature in glue-jupyter is the ability to fade between layers:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')
imviz.load_data('jw01072001001_01101_00005_nrcb1_cal.fits')

imviz.app.data_collection.add_link(WCSLink(imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]'],
                                           imviz.app.data_collection['jw01072001001_01101_00005_nrcb1_cal[SCI]'] ))

viewer1 = imviz.app.get_viewer('viewer-1')
viewer1.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer1.add_data('jw01072001001_01101_00005_nrcb1_cal[SCI]')
viewer1.state.show_axes = False
viewer1.state.layers[0].percentile = 99
viewer1.state.layers[1].percentile = 99

imviz.app

If you go to the layer options and select the second layer in the drop-down and play with the opacity slider you can fade between the two images.

## Blinking

A variation on the above example is that we can set up a button (or in future a keyboard shortcut) to blink between images:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')
imviz.load_data('jw01072001001_01101_00005_nrcb1_cal.fits')

imviz.app.data_collection.add_link(WCSLink(imviz.app.data_collection['jw01072001001_01101_00001_nrcb1_cal[SCI]'],
                                           imviz.app.data_collection['jw01072001001_01101_00005_nrcb1_cal[SCI]'] ))

viewer = imviz.app.get_viewer('viewer-1')
viewer.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer.add_data('jw01072001001_01101_00005_nrcb1_cal[SCI]')
viewer.state.show_axes = False
viewer.state.layers[0].percentile = 99
viewer.state.layers[1].percentile = 99

imviz.app

Click on the following button to see this in action:

In [ ]:
button_blink = Button(description='Blink images')

def blink(event):
    viewer.state.layers[1].visible = not viewer.state.layers[1].visible
    
button_blink.on_click(blink)
button_blink

## Astropy regions

It is already possible to make selections/regions in images and export these to astropy regions:

In [ ]:
imviz = ImViz()
imviz.load_data('jw01072001001_01101_00001_nrcb1_cal.fits')

viewer = imviz.app.get_viewer('viewer-1')
viewer.add_data('jw01072001001_01101_00001_nrcb1_cal[SCI]')
viewer.state.show_axes = False
viewer.state.layers[0].percentile = 99

imviz.app

Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.app.get_subsets_from_viewer('viewer-1')

In [ ]:
regions

Since the region is an astropy region, we can e.g. convert it to a mask:

In [ ]:
mask = regions['Subset 1'].to_mask(mode='exact')

In [ ]:
plt.imshow(mask.to_image((2048, 2048)), origin='lower')